In [1]:
# Первый шаг

# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
from dotenv import load_dotenv
load_dotenv()

API_URL = os.getenv('API_URL')
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')

# Загрузка ads.csv
ads_check = pd.read_csv('ads.csv')

# Загрузка данных из CSV-файлов
visits = pd.read_csv('visits_1k.csv')
regs = pd.read_csv('regs_1k.csv')

# Предварительный просмотр данных
print("Первые строки данных о посещениях:")
display(visits.head())
print("\nПервые строки данных о регистрациях:")
display(regs.head())

# Информация о структуре данных
print("Информация о visits:")
visits.info()
print("\nИнформация о regs:")
regs.info()

# Уникальные значения
print("Уникальные значения в visits['platform']:", visits['platform'].unique())
print("Уникальные значения в regs['platform']:", regs['platform'].unique())
print("Типы регистрации в regs['registration_type']:", regs['registration_type'].unique())

Первые строки данных о посещениях:


,uuid,platform,user_agent,date
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01T13:29:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-01T16:44:28
2,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-03-06T06:12:36
3,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-01T20:16:37
4,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-05T17:42:47



Первые строки данных о регистрациях:


,date,user_id,email,platform,registration_type
0,2023-03-01T00:25:39,8838849,joseph95@example.org,web,google
1,2023-03-01T14:53:01,8741065,janetsuarez@example.net,web,yandex
2,2023-03-01T14:27:36,1866654,robert67@example.org,web,google
3,2023-03-01T02:42:34,1577584,elam@example.net,web,apple
4,2023-03-01T10:27:14,4765395,stephanie68@example.net,web,yandex


Информация о visits:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uuid        1000 non-null   object
 1   platform    1000 non-null   object
 2   user_agent  1000 non-null   object
 3   date        1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB

Информация о regs:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               1000 non-null   object
 1   user_id            1000 non-null   int64 
 2   email              1000 non-null   object
 3   platform           1000 non-null   object
 4   registration_type  1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
Уникальные значения в visits['platform']: ['web' 'android' 'ios']
Уникальные значения в 

In [2]:
# Второй шаг
# Импорт библиотек
import pandas as pd
import os
import requests
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv()
API_URL = os.getenv("API_URL")
DATE_BEGIN = os.getenv("DATE_BEGIN")
DATE_END = os.getenv("DATE_END")

# Запрос к API
visits_url = f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}"
regs_url = f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}"

# Загрузка данных
visits_response = requests.get(visits_url)
visits = pd.DataFrame(visits_response.json())

regs_response = requests.get(regs_url)
regs = pd.DataFrame(regs_response.json())

# Просмотр первых строк
print("Посещения:")
display(visits.head())
print("Регистрации:")
display(regs.head())

# Информация о структурах
print("Информация о visits:")
visits.info()
print("\nИнформация о regs:")
regs.info()

# Количество строк
print("Количество строк в visits:", len(visits))
print("Количество строк в regs:", len(regs))

Посещения:


,visit_id,platform,user_agent,datetime
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01T10:36:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01T06:25:00
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01T10:26:13
3,19322fed-157c-49c6-b16e-2d5cabeb9592,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,2023-03-01T12:33:06
4,04762a22-3c9f-40c9-9ac9-6628c4381836,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01T01:38:35


Регистрации:


,datetime,user_id,email,platform,registration_type
0,2023-03-01T07:40:13,2e0f6bb8-b029-4f45-a786-2b53990d37f1,ebyrd@example.org,web,google
1,2023-03-01T13:14:00,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,knightgerald@example.org,web,email
2,2023-03-01T03:05:50,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,cherylthompson@example.com,web,apple
3,2023-03-01T00:04:47,3e9914e1-5d73-4c23-b25d-b59a3aeb2b60,halldavid@example.org,web,email
4,2023-03-01T18:31:52,27f875fc-f8ce-4aeb-8722-0ecb283d0760,denise86@example.net,web,google


Информация о visits:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263459 entries, 0 to 263458
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   visit_id    263459 non-null  object
 1   platform    263459 non-null  object
 2   user_agent  263459 non-null  object
 3   datetime    263459 non-null  object
dtypes: object(4)
memory usage: 8.0+ MB

Информация о regs:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21836 entries, 0 to 21835
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   datetime           21836 non-null  object
 1   user_id            21836 non-null  object
 2   email              21836 non-null  object
 3   platform           21836 non-null  object
 4   registration_type  21836 non-null  object
dtypes: object(5)
memory usage: 853.1+ KB
Количество строк в visits: 263459
Количество строк в regs: 21836


In [3]:
# Третий шаг

# Удаление ботов
visits_clean = visits[~visits['user_agent'].str.contains('bot', case=False)].copy()

# Преобразование datetime
visits_clean['datetime'] = pd.to_datetime(visits_clean['datetime'])
visits_clean['date_group'] = visits_clean['datetime'].dt.normalize()

regs['datetime'] = pd.to_datetime(regs['datetime'])
regs['date_group'] = regs['datetime'].dt.normalize()

# Удаление дубликатов
visits_clean = visits_clean.drop_duplicates(subset='visit_id')

# Группировка по дате и платформе
visits_grouped = visits_clean.groupby(['date_group', 'platform']).agg(visits=('visit_id', 'count')).reset_index()
regs_grouped = regs.groupby(['date_group', 'platform']).agg(registrations=('user_id', 'count')).reset_index()

# Объединение и расчёт конверсии
conversion_df = pd.merge(visits_grouped, regs_grouped, how='left', on=['date_group', 'platform'])
conversion_df['registrations'] = conversion_df['registrations'].fillna(0).astype(int)
conversion_df['conversion'] = (conversion_df['registrations'] / conversion_df['visits']) * 100
conversion_df = conversion_df.sort_values(by='date_group').reset_index(drop=True)
conversion_df.to_json("conversion.json")

In [4]:
# Четвёртый шаг: Реклама

# Загрузка данных о рекламе
ads = pd.read_csv("ads.csv")

# Преобразование даты
ads['date_group'] = pd.to_datetime(ads['date']).dt.normalize()

# Группировка
ads_grouped = ads.groupby('date_group').agg(
    cost=('cost', 'sum'),
    utm_campaign=('utm_campaign', 'last')
).reset_index()

# Объединение с conversion_df
# Заполнение пропущенных значений
ads_merged = pd.merge(conversion_df, ads_grouped, how='left', on='date_group')
ads_merged['cost'] = ads_merged['cost'].fillna(0).astype(int)
ads_merged['utm_campaign'] = ads_merged['utm_campaign'].fillna('none')

# Сортировка по дате
ads_merged = ads_merged.sort_values(by='date_group').reset_index(drop=True)

# Сохранение
ads_merged.to_json("ads.json")

# Проверка типов и диапазона дат
print(ads_merged['date_group'].dtype)
print(ads_merged['date_group'].min(), ads_merged['date_group'].max())

datetime64[ns]
2023-03-01 00:00:00 2023-08-31 00:00:00


In [5]:
# Первый график: итоговые визиты

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

ads_merged['date_group'] = pd.to_datetime(ads_merged['date_group'])

# Фильтрация по диапазону дат
start_date = pd.to_datetime("2023-03-01")
end_date = pd.to_datetime("2023-09-01")
filtered_data = ads_merged[(ads_merged['date_group'] >= start_date) & (ads_merged['date_group'] <= end_date)]

# Группировка
total_visits = filtered_data.groupby('date_group', as_index=False)['visits'].sum()

# График
plt.figure(figsize=(20, 6))
ax = sns.barplot(data=total_visits, x='date_group', y='visits', color='skyblue')
ax.set_xticks(range(len(total_visits)))
ax.set_xticklabels(total_visits['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=8)

# Подписи над столбиками
for i, row in total_visits.iterrows():
    ax.text(x=i, y=row['visits'] + 10, s=f"{int(row['visits'])}", ha='center', fontsize=7)

# Подписи и сохранение
plt.title("Total Visits")
plt.xlabel("Date")
plt.ylabel("Visits")
plt.tight_layout()
plt.savefig('./charts/total_visits.png')
plt.close()

In [6]:
# Второй график: итоговые регистрации
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

ads_merged['date_group'] = pd.to_datetime(ads_merged['date_group'])

# Фильтрация по нужному периоду
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_data = ads_merged[(ads_merged['date_group'] >= start_date) & (ads_merged['date_group'] <= end_date)]

# Группировка регистраций по дате
total_regs = filtered_data.groupby('date_group', as_index=False)['registrations'].sum()

# График
plt.figure(figsize=(20, 6))
ax = sns.barplot(data=total_regs, x='date_group', y='registrations', color='lightgreen')
ax.set_xticks(range(len(total_regs)))
ax.set_xticklabels(total_regs['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=8)

# Подписи над столбцами
for i, row in total_regs.iterrows():
    ax.text(x=i, y=row['registrations'] + 5, s=f"{int(row['registrations'])}", ha='center', fontsize=7)

# Подписи и сохранение
plt.title("Total Registrations")
plt.xlabel("Date")
plt.ylabel("Registrations")
plt.tight_layout()
plt.savefig('./charts/total_registrations.png')
plt.close()

In [7]:
# Третий график: итоговые визиты с разбивкой по платформам

import matplotlib.pyplot as plt
import pandas as pd

# Фильтрация и преобразование
ads_merged['date_group'] = pd.to_datetime(ads_merged['date_group'])
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_df = ads_merged[(ads_merged['date_group'] >= start_date) & (ads_merged['date_group'] <= end_date)]

# Группировка и поворот таблицы
visits_by_platform = filtered_df.groupby(['date_group', 'platform'], as_index=False)['visits'].sum()
pivot_df = visits_by_platform.pivot(index='date_group', columns='platform', values='visits').fillna(0)

# Построение графика
fig, ax = plt.subplots(figsize=(22, 8))
pivot_df.plot(kind='bar', stacked=True, ax=ax, width=0.9)

# Настройка осей
ax.set_title("Visits by Platform (Stacked)")
ax.set_xlabel("Date")
ax.set_ylabel("Visits")

# Подписи по X по всем дням
ax.set_xticks(range(len(pivot_df.index)))
ax.set_xticklabels(pivot_df.index.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=7)

# Легенда
ax.legend(title="Platform")

# Сохранение
plt.tight_layout()
plt.savefig('./charts/visits_by_platform_stacked.png')
plt.close()

In [8]:
# Четвертый график: итоговые регистрации с разбивкой по платформе

import matplotlib.pyplot as plt
import pandas as pd
import os

# Загрузка и преобразование даты
conversion_df = pd.read_json("conversion.json")
conversion_df['date_group'] = pd.to_datetime(conversion_df['date_group'], unit='ms', errors='coerce')
conversion_df = conversion_df[conversion_df['date_group'].notnull()]

# Фильтрация по нужному диапазону
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
conversion_df = conversion_df[(conversion_df['date_group'] >= start_date) & (conversion_df['date_group'] <= end_date)]

# Проверка наличия данных
if conversion_df.empty:
    raise ValueError("После фильтрации по дате conversion_df пуст. Проверь исходные значения date_group.")

# Группировка и поворот
regs_by_platform = conversion_df.groupby(['date_group', 'platform'], as_index=False)['registrations'].sum()
pivot_df = regs_by_platform.pivot(index='date_group', columns='platform', values='registrations').fillna(0)

# Проверка содержимого
if pivot_df.empty or pivot_df.select_dtypes(include='number').empty:
    raise TypeError("pivot_df пустой или не содержит числовых данных. Проверь исходные значения.")

# Построение графика
fig, ax = plt.subplots(figsize=(22, 8))
pivot_df.plot(kind='bar', stacked=True, ax=ax, width=0.9)

# Настройка осей
ax.set_title("Registrations by Platform (Stacked)")
ax.set_xlabel("Date")
ax.set_ylabel("Registrations")
ax.set_xticks(range(len(pivot_df)))
ax.set_xticklabels(pivot_df.index.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=7)

# Легенда
ax.legend(title="Platform")

# Сохранение
os.makedirs('./charts', exist_ok=True)
plt.tight_layout()
plt.savefig('./charts/registrations_by_platform_stacked.png')
plt.close()

In [9]:
# Пятый график: итоговые конверсии

import matplotlib.pyplot as plt
import pandas as pd
import os

# Загрузка данных
conversion_df = pd.read_json("conversion.json")
conversion_df['date_group'] = pd.to_datetime(conversion_df['date_group'], unit='ms', errors='coerce')
conversion_df = conversion_df[conversion_df['date_group'].notnull()]

# Фильтрация по диапазону
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_df = conversion_df[(conversion_df['date_group'] >= start_date) & (conversion_df['date_group'] <= end_date)]

# Группировка и расчет конверсии
daily = filtered_df.groupby('date_group', as_index=False)[['visits', 'registrations']].sum()
daily['conversion'] = (daily['registrations'] / daily['visits']) * 100

# Построение графика
plt.figure(figsize=(20, 6))
ax = plt.gca()
ax.plot(daily['date_group'], daily['conversion'], marker='o', label='Общая конверсия', color='midnightblue')

# Подписи над точками
for i, row in daily.iterrows():
    ax.text(x=row['date_group'], y=row['conversion'] + 0.7,
            s=f"{int(round(row['conversion'], 0))}%", ha='center', fontsize=7)

# Оформление осей
ax.set_title("Overall Conversion")
ax.set_xlabel("Date")
ax.set_ylabel("Conversion (%)")
ax.set_xticks(daily['date_group'])
ax.set_xticklabels(daily['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=7)

# Остальное оформление
ax.legend()
ax.grid(True, linestyle='--', alpha=0.4)

plt.tight_layout()
os.makedirs('./charts', exist_ok=True)
plt.savefig('./charts/overall_conversion.png')
plt.close()

In [10]:
# Шестой график: конверсия по каждой платформе

import matplotlib.pyplot as plt
import pandas as pd
import os

# Загрузка и обработка данных
conversion_df = pd.read_json("conversion.json")
conversion_df['date_group'] = pd.to_datetime(conversion_df['date_group'], unit='ms', errors='coerce')
conversion_df = conversion_df[conversion_df['date_group'].notnull()]

# Фильтрация по диапазону дат
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_df = conversion_df[(conversion_df['date_group'] >= start_date) & (conversion_df['date_group'] <= end_date)]

# Группировка и расчет конверсий
by_platform = filtered_df.groupby(['date_group', 'platform'], as_index=False)[['visits', 'registrations']].sum()
by_platform['conversion'] = (by_platform['registrations'] / by_platform['visits']) * 100

# Уникальные платформы
platforms = by_platform['platform'].unique()

# Готовим графики
fig, axs = plt.subplots(len(platforms), 1, figsize=(20, 5 * len(platforms)), sharex=True)

# Если одна платформа
if len(platforms) == 1:
    axs = [axs]

# Построение графика по каждой платформе
for i, platform in enumerate(platforms):
    df = by_platform[by_platform['platform'] == platform]
    axs[i].plot(df['date_group'], df['conversion'], marker='o', label=platform, color='teal')

    for j, row in df.iterrows():
        axs[i].text(x=row['date_group'],
                    y=row['conversion'] + 1,
                    s=f"{int(round(row['conversion'], 0))}%",
                    ha='center', fontsize=7)

    axs[i].set_title(f"Conversion for {platform}", fontsize=12)
    axs[i].set_ylabel("Conversion (%)")
    axs[i].legend()
    axs[i].grid(True, linestyle='--', alpha=0.3)

# Оформление общей оси X
axs[-1].set_xlabel("Date")
axs[-1].set_xticks(df['date_group'])
axs[-1].set_xticklabels(df['date_group'].dt.strftime('%Y-%m-%d'), rotation=45, ha='right', fontsize=8)

# Сохранение
plt.tight_layout()
os.makedirs('./charts', exist_ok=True)
plt.savefig('./charts/conversion_by_platform.png')
plt.close()

In [11]:
# Седьмой график: стоимость рекламы

import matplotlib.pyplot as plt
import pandas as pd
import os

# Преобразование и фильтрация
ads_merged['date_group'] = pd.to_datetime(ads_merged['date_group'])
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_df = ads_merged[(ads_merged['date_group'] >= start_date) & (ads_merged['date_group'] <= end_date)]

# Группировка и удаление нулевых значений
daily_cost = filtered_df.groupby('date_group', as_index=False)['cost'].sum()
daily_cost = daily_cost[daily_cost['cost'] > 0].sort_values('date_group')

# Построение графика
plt.figure(figsize=(20, 6))
plt.plot(daily_cost['date_group'], daily_cost['cost'], marker='o', linestyle='-', color='darkorange')

# Подписи значений
for i, row in daily_cost.iterrows():
    plt.text(x=row['date_group'],
             y=row['cost'] + 5,
             s=f"{int(row['cost'])}₽",
             ha='center',
             fontsize=7)

# Настройка осей
plt.title("Aggregated Ad Campaign Costs (by day)")
plt.xlabel("Date")
plt.ylabel("Cost (RUB)")
plt.grid(True, linestyle='--', alpha=0.4)

# Установка xticks с интервалом
xticks = range(0, len(daily_cost), 3)
xticklabels = [date.strftime('%Y-%m-%d') for date in daily_cost['date_group'].iloc[xticks]]
plt.xticks(ticks=daily_cost['date_group'].iloc[xticks], labels=xticklabels, rotation=45, ha='right', fontsize=8)

# Сохранение
os.makedirs('./charts', exist_ok=True)
plt.tight_layout()
plt.savefig('./charts/ad_costs_by_day.png')
plt.close()

In [12]:
# Восьмой график: визиты и регистрации с выделением рекламных кампаний

import matplotlib.pyplot as plt
import pandas as pd
import os

# Приведение даты и фильтрация по периоду
ads_merged['date_group'] = pd.to_datetime(ads_merged['date_group'])
start_date = pd.to_datetime(DATE_BEGIN)
end_date = pd.to_datetime(DATE_END)
filtered_df = ads_merged[(ads_merged['date_group'] >= start_date) & (ads_merged['date_group'] <= end_date)]

# Средние значения
avg_visits = filtered_df['visits'].mean()
avg_regs = filtered_df['registrations'].mean()

# Кампании
campaigns = [
    {
        'name': 'google cpc virtual_reality_workshop',
        'start': '2023-04-10',
        'end': '2023-04-30',
        'color': 'orange'
    },
    {
        'name': 'tg social game_dev_crash_course',
        'start': '2023-03-01',
        'end': '2023-04-08',
        'color': 'steelblue'
    }
]

# Группировка
daily_stats = filtered_df.groupby('date_group', as_index=False)[['visits', 'registrations']].sum()

# Построение
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10), sharex=True)

# Первый график: визиты
ax1.plot(daily_stats['date_group'], daily_stats['visits'], marker='o', color='black', label='Visits')
ax1.axhline(avg_visits, linestyle='--', color='gray', label='Average Visits')

# Второй график: регистрации
ax2.plot(daily_stats['date_group'], daily_stats['registrations'], marker='o', color='forestgreen', label='Registrations')
ax2.axhline(avg_regs, linestyle='--', color='gray', label='Average Registrations')

# Кампании на фоне
for campaign in campaigns:
    for ax in [ax1, ax2]:
        ax.axvspan(pd.to_datetime(campaign['start']),
                   pd.to_datetime(campaign['end']),
                   color=campaign['color'], alpha=0.3)

# Настройка осей и подписей
ax1.set_ylabel("Visits")
ax1.set_title("Visits with Highlighted Campaigns")
ax1.legend()

ax2.set_ylabel("Registrations")
ax2.set_title("Registrations with Highlighted Campaigns")
ax2.legend()

plt.xlabel("Date")
xticks = range(0, len(daily_stats), 3)
xticklabels = [d.strftime('%Y-%m-%d') for d in daily_stats['date_group'].iloc[::3]]
plt.xticks(ticks=daily_stats['date_group'].iloc[xticks], labels=xticklabels, rotation=45, ha='right', fontsize=8)

# Сохранение
os.makedirs('./charts', exist_ok=True)
plt.tight_layout()
plt.savefig('./charts/visits_and_regs_with_campaigns.png')
plt.close()